# List of conflicting CASNumber & IngredientName pairs



In [ ]:
%run "catalog_common.py" 
ID_header('Open-FF:  Conflicting CASNumber/IngredientName pairs', incl_links=True)
set_page_param()

In [ ]:
import pandas as pd
import numpy as np
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
from itables import show as iShow
import itables.options as opt

In [ ]:
df_cas = ana_set.Catalog_set(repo = repo_name, outdir='../common/').get_set(verbose=False)

In [ ]:
len(df_cas[df_cas.bgCAS=='conflictingID'])

In [ ]:
gb = df_cas.groupby(['CASNumber','IngredientName'],as_index=True)['UploadKey'].count().reset_index()
gb = gb.rename({'UploadKey':'record_count'},axis=1)

In [ ]:
df_cas.TradeName = df_cas.TradeName.fillna('missing')
gb2 = df_cas[df_cas.bgCAS=='conflictingID'].groupby(['CASNumber','IngredientName'])['TradeName'].apply(set).reset_index()
gb2
gb3 = df_cas[df_cas.bgCAS=='conflictingID'].groupby(['CASNumber','IngredientName'])['TradeName'].nunique().reset_index()
gb3
gb3.columns = ['CASNumber','IngredientName','TradeName_count']

In [ ]:
casing = grd.get_curated_df(repo_name,'casing_curated.csv')
casing = casing[casing.bgSource=='conflict']
casing = pd.merge(casing,gb,on=['CASNumber','IngredientName'],how='left')
casing = pd.merge(casing,gb2,on=['CASNumber','IngredientName'],how='left')
casing = pd.merge(casing,gb3,on=['CASNumber','IngredientName'],how='left')
#iShow(casing,maxBytes=0)

In [ ]:
ref = grd.get_curated_df(repo_name,'CAS_ref_and_names.csv')
ref.columns = ['curatedCAS','name from CASNumber']
#ref

In [ ]:
casing = pd.merge(casing,ref,on='curatedCAS',how='left')
ref.columns = ['prospect_CAS_fromIng','name from IngNames CAS']
casing = pd.merge(casing,ref,on='prospect_CAS_fromIng',how='left')
#casing

|Explanation of columns in the index|
| :---: |

| Column      | Description |
| :----: | :-------- |
|*CASNumber*| is the text in the CASNumber field of the original FracFocus data set, as found.|
|*IngredientName*| is the text in the IngredientName field of the original FracFocus data set, as found.|
|*bgCAS*| is Open-FF's best guest at the actual CAS registry number for the record. If it is a text value, an authoritative CAS number could not be reliably assigned.
|*bgIngredientName*| is the "primary name" assigned by SciFinder to the bgCAS value.|
|*record_count*| is the number of times this CASNumber/IngredientName pair occurs in the original FracFocus data set.|
|*category*| Open-FF's designation of the type of assignment made.  For example, 'perfect match' means that the original FracFocus CASNumber perfectly matched an authoritative CAS registry number, whereas 'corrected' indicates that Open-FF had to interpret the original information to assign an authoritative registration number.|
|*close_syn* and *comment*| these are notes made during the curation process. |


In [ ]:
gb1 = df_cas.groupby(['CASNumber','IngredientName'],as_index=True)['bgIngredientName'].first().reset_index()

casing = pd.merge(casing,gb1,on=['CASNumber','IngredientName'],how='left')
casing['<CAS'] = '<h1>||</h1>'
casing['<Ing'] = '<h1>||</h1>'
casing.CASNumber = '<b>'+casing.CASNumber+'</b>'
casing.IngredientName = '<b>'+casing.IngredientName+'</b>'
casing['structure_l'] = casing.curatedCAS+'<br>'+casing.curatedCAS.map(lambda x: getChemIDImg(x))
casing['structure_r'] = casing.prospect_CAS_fromIng+'<br>'+casing.prospect_CAS_fromIng.map(lambda x: getChemIDImg(x))
casing.structure_l = '<center>'+casing.structure_l+casing['name from CASNumber']+'</center>'
casing.structure_r = '<center>'+casing.structure_r+casing['name from IngNames CAS']+'</center>'
# casing['curCAS'] = casing.curatedCAS+'<br>'+casing.categoryCAS
# casing['curING'] = casing.prospect_CAS_fromIng+'<br>'+casing.syn_code
# casing['curfinal'] = '<h3>'+casing.bgCAS+'</h3><br>'+casing.bgSource
#casing.comment.fillna('',inplace=True)
iShow(casing[['CASNumber','structure_l','IngredientName','structure_r','record_count','TradeName_count']].reset_index(drop=True))